# DAM

In [1]:
from fastai.vision.all import *
import pandas as pd
import cam
import util

In [2]:
base_model_name = 'densenet121-chexpert-final'

dls, labels = util.chexpert_data_loader(bs=8)

In [3]:
# Load the base model
model_base = cnn_learner(dls, densenet121, n_out=len(labels), path=Path('../saves/'), y_range=(0,1))
model_base.load(base_model_name)

# Create a new densenet121 model
model_dam = cnn_learner(dls, densenet121, n_out=len(labels), y_range=(0,1)).model

# Replace the body of new model with the body from the base model
base_model_body = model_base.model[0].state_dict()
model_dam[0].load_state_dict(base_model_body)

# Set all parameters to require grad
for param in model_dam.parameters():
    param.requires_grad = True
    
# Initialize DAM learner
dam_learner = util.DAM(model_dam, dls, folder=Path('../saves/densenet121-dam'), lr=0.03)

In [4]:
print(f'Initial AUC score: {dam_learner.eval()}')

Initial AUC score: 0.5015548063709537


In [5]:
dam_learner.train(max_epoch=5, lr_div=1.15)

epoch: 0, train_loss: -0.022840, train_auc:0.724548, test_auc:0.779143, lr:0.030000
Update Regularizer @ T=22364!
epoch: 1, train_loss: -0.009245, train_auc:0.717687, test_auc:0.777998, lr:0.026087
Update Regularizer @ T=22364!
epoch: 2, train_loss: -0.005339, train_auc:0.705040, test_auc:0.782418, lr:0.022684
Update Regularizer @ T=22364!
epoch: 3, train_loss: -0.006656, train_auc:0.701617, test_auc:0.779012, lr:0.019725
Update Regularizer @ T=22364!
epoch: 4, train_loss: -0.005125, train_auc:0.694845, test_auc:0.779941, lr:0.017153
Update Regularizer @ T=22364!


In [6]:
dam_learner.opt_func.lr = 0.03
dam_learner.train(max_epoch=2, lr_div=1.15)

epoch: 0, train_loss: -0.000104, train_auc:0.691008, test_auc:0.780079, lr:0.030000
Update Regularizer @ T=22364!


KeyboardInterrupt: 

Get the categorywise AUC scores of the best model.

In [6]:
dam_learner.load('m-epoch 2-2021_May_09-01_59_24.pth.tar')

In [ ]:
print(f'Categorywise AUC score: {dam_learner.eval(average=None)}')